In [72]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 25 14:31:01 2019
@author: minyoungpark

Moved to Jupyter notebook (for better or worse) on 12/03/2020 by Joseph
Updated code as well to streamline 3D reconstruction
"""
import sys

# add calibration_folder to path so that those functions can be used
calib_folder = r'D:\Lab\GIT\proc-joe\Limblab_DLC\cam_calib_20200508'

# set project folder 
project_folder = r'D:\Lab\Data\DLC_videos\Handle_test\Handle_test_20201201-Qiwei-2020-12-01'

# determine if we are using filtered data or not
use_filtered_data = True
remove_triangulation = True

# using reference frame or use an arbitrary frame?
use_reference_frame = False

# imports and other stuff
sys.path.append(calib_folder)

from utils.utils import load_config
from calibration.intrinsic import calibrate_intrinsic
from calibration.extrinsic import calibrate_extrinsic
from triangulation.triangulate import reconstruct_3d
from utils.vis_utils import generate_three_dim_video
from utils.vis_utils import generate_three_dim_pictures
from utils.triangulation_utils import add_static_points
    
import numpy as np
import toml
import glob
import os

In [82]:
# setup config file 
# load basic toml folder and then fill out relevant entries
parsed_toml = toml.load(calib_folder + r'\config_master.toml')

# upate calib video path and prefix and extension
parsed_toml['calibration']['calib_video_path'] = r'D:\Lab\Data\DLC_videos\Handle_test\calib'
parsed_toml['calibration']['calib_video_prefix'] = 'Han_20201123_calib_6in0000'

# update paths to 2d data while removing (or keeping) videos with filtered
if(use_filtered_data):
    vid_list = glob.glob(project_folder + r'\videos\*filtered.csv')
else:
    vid_list_temp = glob.glob(project_folder + r'\videos\*.csv')
    vid_list = []
    for vid_name in vid_list_temp:
        if vid_name.find('filtered') == -1:
            vid_list.append(vid_name)
    
parsed_toml['paths_to_2d_data'] = vid_list
    
# update path to save static data
parsed_toml['path_to_save_static_data'] = project_folder

# update output video path
parsed_toml['output_video_path'] = project_folder + r'\reconstructed-3d-data'

# update triangulation data (or remove if desired)
if(remove_triangulation):
    parsed_toml['triangulation'].pop('axes', None)
    parsed_toml['triangulation'].pop('reference_point', None)
# else, likely leave alone. Could fill this in if we change how we do reference points

# update reconstruction output path and threshold
parsed_toml['triangulation']['reconstruction_threshold'] = 0.7 # 0.7 default
parsed_toml['triangulation']['reconstruction_output_path'] = project_folder + r'\reconstructed-3d-data'

# update labeling scheme and bodyparts interested
# only need to update if using something other than base arm points
parsed_toml['labeling']['scheme'] = []
parsed_toml['labeling']['bodyparts_interested'] = ['handle']

toml_config_file = project_folder + r'\recon_config.toml'

with open(toml_config_file,'w+') as file:
    toml.dump(parsed_toml,file)


In [83]:
config = load_config(toml_config_file)
#%% If you already ran calibration you don't need to run these.
calibrate_intrinsic(config)

calibrate_extrinsic(config)


intrinsics_1.toml already exists.

intrinsics_2.toml already exists.

extrinsics.toml already exists.


In [85]:
if(use_reference_frame):
    labels = ['pointX', 'pointY', 'pointZ']

    snapshots = ['Han_20201123_test_15in00001DLC_resnet50_TestNov23shuffle1_1030000filtered.csv',
                 'Han_20201123_test_15in00001DLC_resnet50_TestNov23shuffle1_1030000filtered.csv']

    static = {'pointX': [[497.1956159,477.1799594],[286.2408452,338.6374849]],
              'pointY': [[602.8473412,420.1615679],[606.4223067,301.0989688]],
              'pointZ': [[493.8415928,182.0259331],[274.6480682,95.18916682]]}
    add_static_points(config, labels, static, snapshots)

if(not os.path.isdir(parsed_toml['triangulation']['reconstruction_output_path'])):
    os.mkdir(parsed_toml['triangulation']['reconstruction_output_path'])
    

    
recovery = reconstruct_3d(config)

100%|███████████████████████████| 1730/1730 [00:00<00:00, 7214.30it/s]


In [ ]:
# =============================================================================
# #%% Save 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# class NumpyArrayEncoder(JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return JSONEncoder.default(self, obj)
# with open("pop_0610_anipose.json", "w") as write_file:
#     json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#     
# #%% Load 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# with open("pop_0317_3.json", "r") as read_file:
#     print("Converting JSON encoded data into Numpy array")
#     recovery = json.load(read_file)
# recovery['registration_mat'] = np.array(recovery['registration_mat'])
# recovery['center'] = np.array(recovery['center'])
# =============================================================================


#%% Save 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
with open("pop_0610_anipose.json", "w") as write_file:
    json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#%% Load 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
with open("pop_0317_3.json", "r") as read_file:
    print("Converting JSON encoded data into Numpy array")
    recovery = json.load(read_file)
recovery['registration_mat'] = np.array(recovery['registration_mat'])
recovery['center'] = np.array(recovery['center'])

#%% generate 3D picture to find the optimal azimuth and elevation value
generate_three_dim_pictures(config)

#%% Try 3D stick figure video

generate_three_dim_video(config)

#%% Testing if the generated 3D results makes sense, by calculating the distance between wrist and hand
"""
import pandas as pd
import numpy as np
from numpy import array as arr

data_path = 'C:/Users/dongq/DeepLabCut/Han-Qiwei-2020-02-21/3D-data/output_3d_data_rotate4.csv'

df = pd.read_csv(data_path)
wrist = np.empty((len(df), 3))
hand = np.empty((len(df), 3))

wrist[:,0] = arr(df['wrist1_x'])
wrist[:,1] = arr(df['wrist1_y'])
wrist[:,2] = arr(df['wrist1_z'])


hand[:,0] = arr(df['hand2_x'])
hand[:,1] = arr(df['hand2_y'])
hand[:,2] = arr(df['hand2_z'])

dist = wrist - hand
dist_finite = dist[np.isfinite(dist[:,0]), :]
dist_3d = np.linalg.norm(dist_finite, axis=1)
print(np.median(dist_3d))
"""